In [3]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
%matplotlib inline

In [4]:
dat = pd.read_csv("car.csv")#Using the car evaluation dataset

### Its a labeled dataset but we will use it for clustering by removing the class label

In [5]:
dat.head()#the first six observations

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [6]:
np.unique(dat['class'])#The data has 4 labels

array(['acc', 'good', 'unacc', 'vgood'], dtype=object)

In [7]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying      1728 non-null object
maint       1728 non-null object
doors       1728 non-null object
persons     1728 non-null object
lug_boot    1728 non-null object
safety      1728 non-null object
class       1728 non-null object
dtypes: object(7)
memory usage: 94.6+ KB


In [8]:
dat.describe()

,buying,maint,doors,persons,lug_boot,safety,class
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,med,med,5more,more,med,med,unacc
freq,432,432,432,576,576,576,1210


In [9]:
#function to calculate the similarity score between two categorical objects
def similarity_score(x,y):
    count = 0
    for i in range(len(x)):
        if x[i]==y[i]:
            count +=1
    return count

### Writing a function called Kmodes it will return a dictionary containing the clustered dataset along with modes,individual cost and total cost

In [10]:
def kmodes(data,clusters,iterations):
    index=np.random.permutation(data.index)[:clusters]#randomly selecting index values for initial k modes
    modes=[]
    labels=[]
    dis=[]
    J=[]
    d=dict()
#calculating the dissimilarity score between each categorical object from the modes and assigning label to minimum score
    for i in index:
        modes.append(np.array(data.ix[i]))#inital k modes added 
    for i in range(len(data)):
        dis=[]
        for j in range(len(modes)):
            dis.append(similarity_score(np.array(data.ix[i]),modes[j]))
        labels.append(np.argmin(dis))
    data['label']=np.array(labels)
#using frequency based method to update the weights
    for i in range(iterations):
        modes=[]
        labels=[]
        for i in np.unique(data['label']):
            modes.append(np.array(data.ix[data['label']==i,:-1].describe().ix[2,:]))
#calculating the diss score between each categorical object and updated modes and assigning new label to minimum score
        for i in range(len(data)):
            dis=[]
            for j in range(len(modes)):
                dis.append(similarity_score(np.array(data.ix[i,:-1]),modes[j]))
            labels.append(np.argmin(dis))
        data['label']=np.array(labels)
#calculating the cost function of individual cluster
    for i in range(len(modes)):
        dis=[]
        for j in range(len(data.ix[data['label']==i,:-1])):
            dis.append(similarity_score(np.array(data.ix[data['label']==i,:-1].iloc[j]),modes[i]))
        J.append(sum(dis))
    d['modes']=modes
    d['costfunction']=J
    d['totalcost']=sum(J)
    d['data']=data
    return d

In [11]:
dic = kmodes(dat.ix[:,:-1],4,2)#calling kmodes on the cars dataset with k equal to 4 as the original had 4 labels. 

In [12]:
dat.ix[:,:-1]

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med
5,vhigh,vhigh,2,2,med,high
6,vhigh,vhigh,2,2,big,low
7,vhigh,vhigh,2,2,big,med
8,vhigh,vhigh,2,2,big,high
9,vhigh,vhigh,2,4,small,low


In [10]:
dic['costfunction']#individual cost function of 4 clusters

[612, 232, 212, 160]

In [11]:
dic['totalcost']#total cost function

1216

In [14]:
cldata = dic['data']#lets see how the clusterd data looks like

### The first 6 observations of each class label

In [26]:
cldata.ix[cldata['label']==0,:].head()

,buying,maint,doors,persons,lug_boot,safety,label
1,vhigh,vhigh,2,2,small,med,0
2,vhigh,vhigh,2,2,small,high,0
4,vhigh,vhigh,2,2,med,med,0
5,vhigh,vhigh,2,2,med,high,0
7,vhigh,vhigh,2,2,big,med,0


In [27]:
cldata.ix[cldata['label']==1,:].head()

,buying,maint,doors,persons,lug_boot,safety,label
0,vhigh,vhigh,2,2,small,low,1
3,vhigh,vhigh,2,2,med,low,1
6,vhigh,vhigh,2,2,big,low,1
9,vhigh,vhigh,2,4,small,low,1
12,vhigh,vhigh,2,4,med,low,1


In [28]:
cldata.ix[cldata['label']==2,:].head()

,buying,maint,doors,persons,lug_boot,safety,label
30,vhigh,vhigh,3,2,med,low,2
32,vhigh,vhigh,3,2,med,high,2
33,vhigh,vhigh,3,2,big,low,2
35,vhigh,vhigh,3,2,big,high,2
39,vhigh,vhigh,3,4,med,low,2


In [29]:
cldata.ix[cldata['label']==3,:].head()

,buying,maint,doors,persons,lug_boot,safety,label
21,vhigh,vhigh,2,more,med,low,3
22,vhigh,vhigh,2,more,med,med,3
31,vhigh,vhigh,3,2,med,med,3
34,vhigh,vhigh,3,2,big,med,3
45,vhigh,vhigh,3,more,small,low,3
